In [100]:
import numpy as np
import hvplot.pandas
import hvplot as hv
import seaborn as sns
import matplotlib.pyplot as plt
import panel as pn
import pandas as pd

#打开文件
df=pd.read_csv('world_wide_COVID_data_daily_data.csv')

#聚类country continents，重复的只取一个
list_of_countries=df['countriesAndTerritories'].to_list()
list_of_countries=sorted(list(set(list_of_countries)))
list_of_continents=df['continentExp'].to_list()
list_of_continents=sorted(list(set(list_of_continents)))

#写模块
countries=pn.widgets.MultiSelect(name='Countries',options=list_of_countries) 
months=pn.widgets.IntRangeSlider(name='Months',start=1, end=12, value=(1,2), step=1)
continents=pn.widgets.MultiSelect(name='Coutinents',options=list_of_continents)
z=pn.widgets.RadioButtonGroup(name="Data to plot", options=['cases', 'deaths'])

#设置关联，画图。按照选定的大洲，国家，选定的月份将特定数据取出来，进行画图，运用.isin()
@pn.depends(continents,watch=True)
def update_countries(continents):
    df_select = df[df['continentExp'].isin(continents)]
    countries.options = sorted(df_select['countriesAndTerritories'].unique().tolist())
@pn.depends(countries,months,z,watch=True)
def make_plot(countries,months,z):
    df_countries=df[df['countriesAndTerritories'].isin(countries)]
    df_countries=df_countries[df_countries['month'].isin(range(months[0],months[1]))]
    plot=df_countries.hvplot(x='dateRep',y=z,width=400)
    return plot

#设置关联，做表。
@pn.depends(countries,months,z,watch=True)
def make_table(countries,months,z):
    df_countries=df[df['countriesAndTerritories'].isin(countries)]
    df_countries=df_countries[df_countries['month'].isin(range(months[0],months[1]))]
    table=df_countries.hvplot.table()
    return table

#输出检验
pn.Row(pn.WidgetBox(continents,countries,months,width=200),make_plot,make_table).show()



Launching server at http://localhost:51302


In [101]:
#template
template=pn.template.FastListTemplate(site='python homework',title='world wide COVID data',
                                       sidebar=["##Settings",continents,countries,months],
                                       main=['main plot',make_plot,make_table]).servable